In [1]:
from __future__ import print_function, division
import os
import time
import torch
import pandas as pd
import numpy as np
# For showing and formatting images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# For importing datasets into pytorch
import torchvision.datasets as dataset

# Used for dataloaders
from torch.utils.data import DataLoader

# For pretrained resnet34 model
import torchvision.models as models

# For optimisation function
import torch.nn as nn
import torch.optim as optim

# For turning data into tensors
import torchvision.transforms as transforms

# For loss function
import torch.nn.functional as F

# Tensor to wrap data in
from torch.autograd import Variable

In [2]:
PATH = '/home/cell/data/plant_seedlings/model/'
!ls {PATH+"train"}

Black-grass  Common Chickweed  Loose Silky-bent   Shepherds Purse
Charlock     Common wheat      Maize		  Small-flowered Cranesbill
Cleavers     Fat Hen	       Scentless Mayweed  Sugar beet


In [3]:
batch_size = 16
sz = 224

In [4]:
## Image loaders
## Dataset transforms puts the images in tensor form
normalise = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_raw = dataset.ImageFolder(PATH+"train", transform=transforms.Compose([transforms.RandomResizedCrop(sz),
                                                                            transforms.ToTensor(),
                                                                           normalise]))
train_loader = DataLoader(train_raw, batch_size=batch_size, shuffle=True, num_workers=4)

valid_raw = dataset.ImageFolder(PATH+"valid", transform=transforms.Compose([transforms.CenterCrop(sz),
                                                                            transforms.ToTensor(),
                                                                           normalise]))
valid_loader = DataLoader(valid_raw, batch_size=batch_size, shuffle=False, num_workers=4)

In [5]:
## Create resnet model
resnet34=models.resnet34(pretrained=True)

## Loss function and optimiser
criterion = nn.CrossEntropyLoss().cuda()
optimiser = optim.Adam(resnet34.fc.parameters(), lr=0.001, weight_decay=0.001)

In [6]:
def train(epochs):
    #epoch=1
    resnet34.train()
    time_secs = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        start_time = time.time()
        #print(batch_idx)
        data, target = Variable(data), Variable(target)
        optimiser.zero_grad()
        output = resnet34(data)
        loss=criterion(output, target)
        loss.backward()
        optimiser.step()
        time_secs += (time.time() - start_time)
        if batch_idx % 10 == 0:
            print("Run time for 10 batches was: ", time_secs)
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data[0]))
            time_secs = 0
            #break;

In [7]:
def validation():
    resnet34.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile = True), Variable(target)
        output=resnet34(data)
        test_loss += criterion(output, target).data[0]
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(valid_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(valid_loader.dataset),
    100. * correct / len(valid_loader.dataset)))

In [8]:
## Loop through epochs training data and then testing it
for epoch in range(1,10):
    train(epoch)
    validation()

Run time for 10 batches was:  7.7427427768707275
Train epoch: 1 [0/3806 (0%)]	Loss: 9.504869
Run time for 10 batches was:  66.25943541526794
Train epoch: 1 [160/3806 (4%)]	Loss: 4.794926
Run time for 10 batches was:  65.76920747756958
Train epoch: 1 [320/3806 (8%)]	Loss: 3.903837
Run time for 10 batches was:  65.48485612869263
Train epoch: 1 [480/3806 (13%)]	Loss: 3.400758
Run time for 10 batches was:  64.98099827766418
Train epoch: 1 [640/3806 (17%)]	Loss: 2.331260
Run time for 10 batches was:  65.25183844566345
Train epoch: 1 [800/3806 (21%)]	Loss: 3.052902
Run time for 10 batches was:  65.10970306396484
Train epoch: 1 [960/3806 (25%)]	Loss: 3.012307
Run time for 10 batches was:  64.9295711517334
Train epoch: 1 [1120/3806 (29%)]	Loss: 2.803811
Run time for 10 batches was:  65.06325435638428
Train epoch: 1 [1280/3806 (34%)]	Loss: 2.539026
Run time for 10 batches was:  64.72999000549316
Train epoch: 1 [1440/3806 (38%)]	Loss: 2.780199
Run time for 10 batches was:  65.13281917572021
Trai

Run time for 10 batches was:  63.041414976119995
Train epoch: 4 [2080/3806 (55%)]	Loss: 1.032371
Run time for 10 batches was:  62.733288526535034
Train epoch: 4 [2240/3806 (59%)]	Loss: 1.704320
Run time for 10 batches was:  62.28198742866516
Train epoch: 4 [2400/3806 (63%)]	Loss: 1.238079
Run time for 10 batches was:  62.45811414718628
Train epoch: 4 [2560/3806 (67%)]	Loss: 1.115751
Run time for 10 batches was:  61.95737075805664
Train epoch: 4 [2720/3806 (71%)]	Loss: 1.256768
Run time for 10 batches was:  62.208385944366455
Train epoch: 4 [2880/3806 (76%)]	Loss: 1.208915
Run time for 10 batches was:  61.88788318634033
Train epoch: 4 [3040/3806 (80%)]	Loss: 1.053379
Run time for 10 batches was:  61.77302360534668
Train epoch: 4 [3200/3806 (84%)]	Loss: 1.110067
Run time for 10 batches was:  62.416311264038086
Train epoch: 4 [3360/3806 (88%)]	Loss: 1.392629
Run time for 10 batches was:  62.390942096710205
Train epoch: 4 [3520/3806 (92%)]	Loss: 1.059729
Run time for 10 batches was:  62.13

Run time for 10 batches was:  63.123897314071655
Train epoch: 8 [160/3806 (4%)]	Loss: 1.135716
Run time for 10 batches was:  63.100257396698
Train epoch: 8 [320/3806 (8%)]	Loss: 1.102621
Run time for 10 batches was:  62.973812103271484
Train epoch: 8 [480/3806 (13%)]	Loss: 0.786348
Run time for 10 batches was:  63.01951885223389
Train epoch: 8 [640/3806 (17%)]	Loss: 2.178388
Run time for 10 batches was:  62.76961588859558
Train epoch: 8 [800/3806 (21%)]	Loss: 0.804844
Run time for 10 batches was:  62.86982703208923
Train epoch: 8 [960/3806 (25%)]	Loss: 1.095928
Run time for 10 batches was:  62.62620949745178
Train epoch: 8 [1120/3806 (29%)]	Loss: 0.873884
Run time for 10 batches was:  62.786614418029785
Train epoch: 8 [1280/3806 (34%)]	Loss: 1.027603
Run time for 10 batches was:  62.75418972969055
Train epoch: 8 [1440/3806 (38%)]	Loss: 1.337286
Run time for 10 batches was:  63.024214029312134
Train epoch: 8 [1600/3806 (42%)]	Loss: 0.874333
Run time for 10 batches was:  62.8167324066162